# Convolutional Neural Net(합성곱 신경망, CNN)

In [77]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module): 
    # 일반적으로 class를 이용해 neural net 모형을 정의합니다.
    # class의 개념에 익숙하지 않으신 분들은 https://wikidocs.net/28 여기를 참고하세요.
    def __init__ (self):
        super(Net, self).__init__() #super는 슈퍼 클래스의 method를 호출합니다.
        # 1 input image channel, 6 output channels, 5x5 square convolution
        
        # kernel
        self.conv1 = nn.Conv2d(1,6,5) # 1,6,5는 각각 in_channels, out_channels, kernel_size를 뜻합니다. filter 6개를 씁니다.
        self.conv2 = nn.Conv2d(6,16,5) # 6으로 나왔으니 6으로 받고 16으로 나갑니다, kernel_size는 5입니다.
        #kernel_size는 convolving하는 kernel의 크기를 뜻합니다. kernel = filter입니다. 여기서는 5x5 겠죠?
        
        # an affine operation: y = Wx +b
        self.fc1 = nn.Linear(16*5*5,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    
    def forward(self,x):
        x = self.conv1(x)
        x = F.relu(x)
        # Max pooling over a (2,2) window
        x = F.max_pool2d(x,(2,2)) #input = x, kernel_size = 2x2, stride = None, stirde=None이면 kernel_size를 따라갑니다. stride=2
        
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x,2)
        
        # -1은 몇개가 들어올 지 모른다는 뜻입니다.
        # num_flat_features는 1x? 형태로 flat하게 reshape합니다.
        x = x.view(-1, self.num_flat_features(x))
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    # x.size()[0]은 input 갯수이기 때문에 제외합니다. 각각의 input값들을 1x?형태로 reshape합니다.
    # torch.randn(5,3,3) 을 생각해보면 개수를 의미하는 5를 제외하고 3x3 = 9의 값을 return합니다.

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


- forward 이후에 backward는 autograd를 이용해 자동의로 정의할 수 있습니다.
- net의 weight들은 net.parameters()에 의해 return됩니다.

In [78]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) #conv1의 weight 벡터 사이즈입니다. 5x5 행렬 6개겠죠? print(params)로 확인해보세요

10
torch.Size([6, 1, 5, 5])


In [79]:
input = torch.randn(1,1,32,32) #32x32 하나를 input으로 넣어줍니다. 차원에 유의하세요.

out = net(input) 
print(out)

tensor([[ 0.0367, -0.0806,  0.0891, -0.0654, -0.0138, -0.0105, -0.0813, -0.0589,
         -0.0416,  0.1057]], grad_fn=<ThAddmmBackward>)


input이 왜 1x1x32x32로 들어갈까요? 이유는 간단합니다. torch.nn에서는 mini-batch만 지원합니다.  
따라서 nnconv2d는 nsamples x nChannels x Height x Width의 4차원 tensor를 입력으로 합니다.  
1개의 값을 input channel(1)에 넣으니 1x1이 됩니다.

In [80]:
net.zero_grad()
out.backward(torch.randn(1,10)) # loss를 정의하지 않았으므로 임의의 값으로 back prop을 합니다.

torch.tensor는 autograd 연산을 지원합니다. 또한 tensor의 gradient를 갖고 있습니다. back prop할때마다 이를 초기화 해야 합니다.  
nn.module은 weight를 캡슐화해서 GPU로의 이동, 내보내기, 불러오기 등의 작업을 도와줍니다.  
nn.parameter는 tensor의 종류로 module에 할당될 때 자동으로 weight로 등록됩니다.

## Loss function(손실 함수)

손실함수는 output, target을 한 쌍의 입력으로 받아 ouput이 target으로부터 얼마나 떨어져 있는지를 추정하는 값을 계산합니다.  
output은 net이 계산한 추정값(출력), target은 실제 값(정답)입니다.

In [81]:
output = net(input)
target = torch.arange(1,11, dtype = torch.float)
target = target.view(1,-1) # reshape same as torch.unsqueeze(target, dim=0)
criterion = nn.MSELoss()

loss = criterion(output,target)
print(loss)

tensor(38.6260, grad_fn=<MseLossBackward>)


loss.grad_fn는 다음과 같은 연산 그래프를 따릅니다.  

input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d  
-> view -> linear -> relu -> linear -> relu -> linear  
-> MSELoss  
-> loss  

In [82]:
print(loss.grad_fn) # MSELoss
print(loss.grad_fn.next_functions[0][0]) #Linear
print(loss.grad_fn.next_functions[0][0].next_functions[1][0]) #relu
print(loss.grad_fn.next_functions[0][0].next_functions[1][0].next_functions[0][0]) #Linear
print(loss.grad_fn.next_functions[0][0].next_functions[1][0].next_functions[0][0].next_functions[1][0]) #relu
print(loss.grad_fn.next_functions[0][0].next_functions[1][0].next_functions[0][0].next_functions[1][0].next_functions[0][0]) #Linear
a = loss.grad_fn.next_functions[0][0].next_functions[1][0].next_functions[0][0].next_functions[1][0].next_functions[0][0]
print(a.next_functions[1][0]) #view
print(a.next_functions[1][0].next_functions[0][0]) #maxpool2d

## 역전파

In [87]:
net.zero_grad() # 각 연산의 gradient가 저장되어 있는데 그 값을 초기화 해줍니다.

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

# 보다시피 값이 모두 0으로 초기화되어 있습니다.

loss.backward() # gradient를 계산합니다.

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)
print(net.conv2.bias.grad)
print(net.fc1.bias.grad)
print(net.fc2.bias.grad)
print(net.fc3.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0080, -0.0462,  0.0108,  0.0174,  0.0168, -0.0126])
tensor([ 0.0803,  0.0054, -0.1227, -0.0187, -0.0357,  0.0007,  0.0455, -0.0923,
         0.0324,  0.1421,  0.0000, -0.0450,  0.0514,  0.0545, -0.0093, -0.0722])
tensor([ 0.0807,  0.0518, -0.0182,  0.0000,  0.0000,  0.0462, -0.1344,  0.0527,
         0.0000,  0.0000,  0.0686,  0.1070,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000, -0.0323, -0.1667, -0.1134,  0.0000,  0.0046,  0.0000,  0.0000,
         0.0404,  0.0000,  0.0000,  0.0000,  0.0000, -0.0142,  0.0000,  0.0655,
         0.0000,  0.0000,  0.0596, -0.0923,  0.0000,  0.0000,  0.0000, -0.0731,
         0.0239,  0.0898,  0.0051,  0.0000,  0.0000,  0.0000, -0.0244,  0.0000,
         0.0000,  0.0000,  0.0094,  0.1050,  0.0000,  0.1050,  0.0000,  0.1596,
         0.0003, -0.1304, -0.0382, -0.0671,  0.0000,  0.0000,  0.0000, -0.0119,
         0.0000,  0.0000, -0.0717,  0.00

## weight update(가중치 갱신)

- 가중치 갱신은 torch.optim 패키지를 이용해 이루어집니다.

In [88]:
import torch.optim as optim

# optimizer를 생성합니다
optimizer = optim.Adam(net.parameters(),lr=0.01) #Adam optimizer를 사용합니다. learning rate는 0.01입니다.

# 학습 과정(가중치 갱신 과정)은 다음과 같습니다.
optimizer.zero_grad() # 기존의 변화도에 대해 누적되는 것을 막기 위해 zero_grad()로 초기화합니다.
output = net(input)
loss = criterion(output,target)
loss.backward()
optimizer.step()